In [21]:
# Assemble a list of the similar events from the CC detection results

## Import modules

from obspy import read, UTCDateTime, Stream
from cc_utils import template_match_3c
from preproc_utils import preprocess_stream
from snuffler_utils import read_picks
from os.path import join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
from glob import glob

In [22]:
## Inputs
root = "/Volumes/OmanData/geophones_no_prefilt/data"
timewin_tmp = "2020-01-12-17-00-00"
tempname = "Example2"
numdet_min = 4 # minimum number of detections to be considered a similar event
delta_max = 0.05 # maximum time difference to be considered a similar event

In [23]:
## Read the detection results and combine them into a single data frame
pattern = join(root, timewin_tmp, f"CCdetections_{tempname}*")
inpaths = glob(pattern)

detdfs = []
for inpath in inpaths:
    detdf = pd.read_csv(inpath, delimiter=" ", index_col=0)
    detdfs.append(detdf)

detdf = pd.concat(detdfs, ignore_index=True)
detdf.sort_values(by="peak_time", inplace=True, ignore_index=True)
numdet = len(detdf)

print(f"There are in total {numdet} detections.")

There are in total 504 detections.


In [24]:
## Find the similar events
events = []
i = 0
while i < numdet:
    stname0 = detdf["station"][i]
    tshift0 = detdf["time_shift"][i]
    numdet_ev = 1
    j = i + 1

    while j < numdet:
        stname = detdf["station"][j]
        tshift = detdf["time_shift"][j]

        if tshift - tshift0 > delta_max:
            break
        else:
            if stname != stname0:
                numdet_ev += 1
        
        j += 1

    # print(numdet_ev)
    if numdet_ev >= numdet_min:
        evdf = detdf.iloc[i:j]
        evdf.reset_index(drop=True, inplace=True)
        # print(evdf)
        events.append(evdf)
        # detdf.drop(detdf.index[i:j], inplace=True)
        i = j
    else:
        i += 1

    # numdet = len(detdf)
    
numev = len(events)
print(f"There are in total {numev} events similar to the template.")
    


There are in total 32 events similar to the template.


In [25]:
## Save the similar events

for i, event in enumerate(events):
    print(event)
    outpath = join(root, timewin_tmp, f"{tempname}_Match{i+1}.csv")
    event.to_csv(outpath, sep=" ")
    print(f"Saved to {outpath}")
    
    

   template station  cc_value  left_thresholds  right_thresholds  \
0  Example2     A04  0.787489         0.658232          0.249242   
1  Example2     A05  0.717141         0.543062          0.222625   
2  Example2     A02  0.764959         0.399263          0.179071   
3  Example2     A06  0.715493         0.213712          0.179279   

                       peak_time    time_shift  duration  amplitude_ratio_z  \
0  2020-01-12 13:12:46.361046774 -15049.001039      0.05           0.422801   
1  2020-01-12 13:12:46.365046775 -15049.001039      0.05           0.719405   
2  2020-01-12 13:12:46.370046775 -15049.001039      0.05           0.683490   
3  2020-01-12 13:12:46.377046775 -15049.001039      0.05           0.474992   

   amplitude_ratio_1  amplitude_ratio_2  
0           0.719528           0.631262  
1           0.492038           0.308494  
2           0.631150           0.187641  
3           0.454981           0.102697  
Saved to /Volumes/OmanData/geophones_no_prefilt/data/